In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from dataset import Dataset
import numpy as np

In [8]:
from common import *

In [10]:
data = Dataset(name='Diagnosis', dirname='preprocess/Diagnosis/')

INFO - 2019-02-11 13:50:53,637 - Reading Structured data ...
INFO - 2019-02-11 13:50:54,149 - Reading Notes ...
INFO - 2019-02-11 13:51:04,020 - Stratifying ...


In [11]:
labellist = [x for x in data.dataframe.columns if x.startswith('y_')]
data.generate_labels(labellist, len(labellist), 'multilabel')

In [12]:
data.generate_encoded_field('gender_y', 'onehot')
data.generate_encoded_field('age_y', 'onehot')
data.generate_encoded_field('ethnicity_y', 'onehot')

In [13]:
features = [x for x in data.dataframe.columns if x.startswith('feature')]
for f in features :
    data.generate_encoded_field(f, 'trivial')
    
data.set_structured_params(regexs=[r'^feature', 'gender_y', 'age_y', 'ethnicity_y'])

In [14]:
from trainer import Trainer, Evaluator

In [15]:
from models.Vanilla import ClassificationTrainer as BasicCT
from models.Hierarchical import ClassificationTrainer as HierCT

In [16]:
from Experiments.experiments import experiments, hierarchical_experiments, structured_experiments

Basic Experiments
=================

In [19]:
train_data = data.get_data('train', structured=True)
dev_data = data.get_data('dev', structured=True)
test_data = data.get_data('test', structured=True)

Pos Percentage [0.21358209 0.07716941 0.10244004 0.31971179 0.13028647 0.13014724
 0.20870897 0.07160013 0.26850917 0.31939852 0.09485189 0.19033033
 0.28253681 0.41581677 0.26697762 0.07180897 0.13039089 0.0897003
 0.04953183 0.04076021 0.08917818 0.14152947 0.18249852 0.14615893
 0.07797   ]
Pos Percentage [0.20548768 0.06969758 0.10194272 0.31744442 0.12237132 0.13458842
 0.2032846  0.06649309 0.25776087 0.31484078 0.0929301  0.19587422
 0.29541358 0.42339275 0.25475666 0.06789505 0.12096936 0.08531945
 0.0530743  0.03745243 0.0817144  0.1327859  0.17684759 0.13198478
 0.07790907]
Pos Percentage [0.20977494 0.06768559 0.10732281 0.32129661 0.12512597 0.12395029
 0.21229426 0.06902922 0.26603964 0.32935841 0.09103124 0.19146792
 0.28081962 0.42122943 0.26184078 0.07910648 0.12428619 0.08834397
 0.05424924 0.04165267 0.09271078 0.13671481 0.17954316 0.14124958
 0.08095398]


In [20]:
train_data = data.filter_data_length(train_data, truncate=90)
dev_data = data.filter_data_length(dev_data, truncate=90)

INFO - 2019-02-11 13:56:47,520 - Maximum Sentence Length 721825.000000 , 90 percentile length 16714.400000 ... 
INFO - 2019-02-11 13:56:47,527 - Pos Percentage of remaining data ... 
INFO - 2019-02-11 13:56:47,547 - [0.19322401 0.07650062 0.10272277 0.3112237  0.12059097 0.12650835
 0.20064975 0.07023515 0.25978496 0.32545637 0.09316986 0.18722927
 0.28480817 0.42090811 0.24957457 0.06973236 0.1237237  0.08249536
 0.04641089 0.03774752 0.08048422 0.12349165 0.15741027 0.12925433
 0.06845606]
INFO - 2019-02-11 13:56:48,531 - Maximum Sentence Length 407062.000000 , 90 percentile length 16420.400000 ... 
INFO - 2019-02-11 13:56:48,532 - Pos Percentage of remaining data ... 
INFO - 2019-02-11 13:56:48,536 - [0.18517694 0.06877365 0.10549744 0.30847986 0.11395504 0.12819942
 0.19719564 0.06543512 0.24905408 0.32383708 0.09014022 0.19519252
 0.30202537 0.42777654 0.23213888 0.06209659 0.1135099  0.07834409
 0.04941019 0.03360783 0.07589584 0.11551302 0.1502337  0.11662586
 0.06855108]


In [ ]:
for e in experiments :
    config = e(data, structured=True)
    print(config)
    trainer = Trainer(BasicCT, config, _type='multilabel')
    trainer.train(train_data, dev_data, save_on_metric='macro_roc_auc')

    evaluator = Evaluator(BasicCT, trainer.model.dirname, _type='multilabel')
    _ = evaluator.evaluate(dev_data, save_results=True)
    print('='*300)

INFO - 2019-02-11 13:57:06,380 - instantiating class <class 'models.Model.Model'> from params {'type': 'seq_classifier', 'embedder': {'type': 'token_embedder', 'vocab_size': 28172, 'embed_size': 200, 'embedding_file': 'preprocess/Diagnosis//embedding_matrix.npy'}, 'decoder': {'num_layers': 2, 'hidden_dims': [128, 25], 'activations': ['tanh', 'linear']}, 'predictor': {'type': 'multilabel'}, 'structured': {'use_structured': True, 'structured_dim': 142}, 'encoder': {'type': 'lstm', 'hidden_size': 128}} and extras {}
INFO - 2019-02-11 13:57:06,380 - type = seq_classifier
INFO - 2019-02-11 13:57:06,381 - instantiating class <class 'models.modules.Embedder.Embedder'> from params {'type': 'token_embedder', 'vocab_size': 28172, 'embed_size': 200, 'embedding_file': 'preprocess/Diagnosis//embedding_matrix.npy'} and extras {}
INFO - 2019-02-11 13:57:06,381 - embedder.type = token_embedder
INFO - 2019-02-11 13:57:06,382 - instantiating class <class 'models.modules.Embedder.TokenEmbedder'> from par

{'model': {'type': 'seq_classifier', 'embedder': {'type': 'token_embedder', 'vocab_size': 28172, 'embed_size': 200, 'embedding_file': 'preprocess/Diagnosis//embedding_matrix.npy'}, 'decoder': {'num_layers': 2, 'hidden_dims': [128, 25], 'activations': ['tanh', 'linear']}, 'predictor': {'type': 'multilabel'}, 'structured': {'use_structured': True, 'structured_dim': 142}, 'encoder': {'type': 'lstm', 'hidden_size': 128}}, 'training_config': {'type': 'Adam', 'groups': [('.*', {'lr': 0.001, 'weight_decay': 1e-05})], 'common': {'bsize': 32, 'class_weight': True}}, 'exp_config': {'exp_name': 'Diagnosis/Basic/LSTM(hs=128)+Structured'}}
Setting Embedding


{'macro_roc_auc': 0.776668436481456, 'macro_pr_auc': 0.3801454812607655}


,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8,label_9,label_10,label_11,label_12,label_13,label_14,label_15,label_16,label_17,label_18,label_19,label_20,label_21,label_22,label_23,label_24
1/f1-score,0.476,0.496,0.335,0.555,0.364,0.361,0.401,0.217,0.543,0.646,0.449,0.496,0.535,0.607,0.482,0.210,0.368,0.242,0.145,0.109,0.206,0.381,0.539,0.394,0.342
1/precision,0.372,0.358,0.243,0.475,0.243,0.241,0.285,0.127,0.452,0.580,0.307,0.378,0.461,0.572,0.362,0.122,0.251,0.147,0.080,0.058,0.121,0.255,0.420,0.265,0.219
1/recall,0.660,0.809,0.542,0.667,0.727,0.722,0.678,0.738,0.679,0.729,0.832,0.723,0.637,0.647,0.721,0.753,0.692,0.699,0.730,0.762,0.683,0.751,0.753,0.773,0.782
1/support,832.000,309.000,474.000,1386.000,512.000,576.000,886.000,294.000,1119.000,1455.000,405.000,877.000,1357.000,1922.000,1043.000,279.000,510.000,352.000,222.000,151.000,341.000,519.000,675.000,524.000,308.000


Model not saved on  macro_roc_auc 0.776668436481456


...

{'model': {'type': 'seq_classifier_with_attention', 'embedder': {'type': 'token_embedder', 'vocab_size': 28172, 'embed_size': 200, 'embedding_file': 'preprocess/Diagnosis//embedding_matrix.npy'}, 'decoder': {'num_layers': 2, 'hidden_dims': [128, 25], 'activations': ['tanh', 'linear']}, 'predictor': {'type': 'multilabel'}, 'structured': {'use_structured': True, 'structured_dim': 142}, 'encoder': {'type': 'lstm', 'hidden_size': 128}, 'attention': {'similarity': {'type': 'additive', 'hidden_size': 128}}}, 'training_config': {'type': 'Adam', 'groups': [('.*', {'lr': 0.001, 'weight_decay': 1e-05})], 'common': {'bsize': 32, 'class_weight': True}}, 'exp_config': {'exp_name': 'Diagnosis/Attention/LSTM(hs=128)+Attention(additive)(hs=128)+Structured'}}
Setting Embedding


INFO - 2019-02-11 16:09:11,465 - instantiating class <class 'models.modules.Predictor.Predictor'> from params {'type': 'multilabel'} and extras {}
INFO - 2019-02-11 16:09:11,466 - predictor.type = multilabel
INFO - 2019-02-11 16:09:11,466 - instantiating class <class 'models.modules.Predictor.Multilabel_predictor'> from params {} and extras {}


{'macro_roc_auc': 0.8283013565837313, 'macro_pr_auc': 0.49967581056239324}


,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8,label_9,label_10,label_11,label_12,label_13,label_14,label_15,label_16,label_17,label_18,label_19,label_20,label_21,label_22,label_23,label_24
1/f1-score,0.507,0.596,0.377,0.588,0.446,0.376,0.422,0.281,0.582,0.717,0.436,0.488,0.563,0.623,0.502,0.266,0.466,0.350,0.172,0.145,0.212,0.429,0.566,0.453,0.323
1/precision,0.368,0.475,0.238,0.525,0.305,0.259,0.295,0.170,0.456,0.614,0.285,0.358,0.478,0.609,0.389,0.158,0.325,0.225,0.099,0.081,0.126,0.290,0.435,0.304,0.196
1/recall,0.812,0.799,0.914,0.669,0.828,0.686,0.738,0.813,0.805,0.862,0.928,0.765,0.684,0.637,0.706,0.839,0.822,0.787,0.653,0.702,0.674,0.821,0.809,0.893,0.922
1/support,832.000,309.000,474.000,1386.000,512.000,576.000,886.000,294.000,1119.000,1455.000,405.000,877.000,1357.000,1922.000,1043.000,279.000,510.000,352.000,222.000,151.000,341.000,519.000,675.000,524.000,308.000


Model not saved on  macro_roc_auc 0.8283013565837313


In [ ]:
for e in hierarchical_experiments :
    config = e(data, structured=False)
    print(config)
    trainer = Trainer(HierCT, config)
    trainer.train(train_data, dev_data)

    evaluator = Evaluator(HierCT, trainer.model.dirname)
    _ = evaluator.evaluate(dev_data, save_results=True)
    print('='*30)

In [ ]:
train_data = data.get_data('train', structured=True, encodings=data.structured_columns)
dev_data = data.get_data('dev', structured=True, encodings=data.structured_columns)
train_data = data.filter_data_length(train_data, truncate=90)
dev_data = data.filter_data_length(dev_data, truncate=90)

for e in structured_experiments :
    config = e(data, structured=True, encodings=data.structured_columns)
    print(config)
    
    trainer = Trainer(BasicCT, config)
    trainer.train(train_data, dev_data)

    evaluator = Evaluator(BasicCT, trainer.model.dirname)
    _ = evaluator.evaluate(dev_data, save_results=True)
    print('='*30)

Baselines
==========

In [ ]:
train_data = data.get_data('train', structured=True)
dev_data = data.get_data('dev', structured=True)

In [ ]:
from PatientVec.models.baselines.LR import LR, LDA

In [ ]:
lr = LR({'vocab' : data.vocab, 'stop_words' : True, 'exp_name' : data.name})
lr.train(train_data)
lr.evaluate(dev_data, save_results=True)
# lr.get_features(n=20)

In [ ]:
lda = LDA({'vocab' : data.vocab, 'stop_words' : True, 'exp_name' : data.name})
lda.train(train_data)
lda.evaluate(dev_data, save_results=True)
print(lda.get_topics(n=10))
topics = lda.get_topics(n=10)
print([topics[i] for i in np.argsort(lda.lda_classifier.coef_[0])])

In [ ]:
[topics[i] for i in np.argsort(lda.lda_classifier.coef_[0])]

In [ ]:
preds = outputs['predictions'][:, 0]

In [ ]:
from common import collapse_and_print_word_attn, print_sent_attn

In [ ]:
n = 50
collapse_and_print_word_attn(data.vocab, dev_data.X[n], outputs['word_attentions'][n])
print_sent_attn(data.vocab, dev_data.X[n], outputs['sentence_attentions'][n])

In [ ]:
outputs['sentence_attentions'][0]

In [ ]:
from scipy.stats import spearmanr, kendalltau

In [ ]:
corrs = [kendalltau(range(len(outputs['sentence_attentions'][i])), outputs['sentence_attentions'][i]) 
         for i in range(len(outputs['sentence_attentions']))]

In [ ]:
rho, leng = zip(*[(x[0], y) for x, y in zip(corrs, [len(z) for z in outputs['sentence_attentions']]) if x[0] == x[0]])

In [ ]:
plt.hist(rho, bins=30)

In [ ]:
pval, leng1 = zip(*[(x[1], y) for x, y in zip(corrs, [len(z) for z in outputs['sentence_attentions']]) if x[1] == x[1]])

Saving Models
==============

In [ ]:
import os
for e in experiments :
    config = e(data, structured=False)
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
    config = e(data, structured=True)
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
for e in hierarchical_experiments :
    config = e(data, structured=False)
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/hierarchical_classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
    config = e(data, structured=True)
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/hierarchical_classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
for e in structured_experiments :
    config = e(data, structured=True, encodings=['gender_y', 'ethnicity_y', 'age_y'])
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
    config = e(data, structured=False, encodings=['gender_y', 'ethnicity_y', 'age_y'])
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
    config = e(data, structured=True, encodings=data.structured_columns)
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
    config = e(data, structured=False, encodings=data.structured_columns)
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
    
for e in os.listdir('outputs/baselines/Readmission_y/baselines/') :
    filename = os.path.join('outputs/baselines/Readmission_y/baselines/', e)
    push_latest_model(filename, os.path.join('Readmission_y/baselines/', e))

In [ ]:
dataset = 'Readmission_y'
dataset_path = os.path.join('latex_evals', dataset)
output_path = os.path.join('Text-encoding-EHR/results/', dataset)
os.makedirs(output_path, exist_ok=True)

In [ ]:
dirs = os.listdir(dataset_path)
keys_to_use = ['1/precision', '1/recall', '1/f1-score', 'accuracy', 'roc_auc', 'pr_auc']
for d in dirs :
    subpath = os.path.join(dataset_path, d)
    output_file = os.path.join(output_path, d + '.csv')
    dfs = []
    for f in sorted(os.listdir(subpath)) :
        if os.path.isfile(os.path.join(subpath, f)) :
            d = json.load(open(os.path.join(subpath, f)))
            results = {k:d['results'][k] for k in keys_to_use}
            results['Method'] = f[:-14].replace('+', ' +').replace('_', ':')
            dfs.append(pd.DataFrame([results]))
        else :
            logging.error("%s not a file", f)
    
    dfs = pd.concat(dfs)
    dfs.to_csv(output_file, columns=['Method'] + keys_to_use, index=False)